In [1]:
import xml.etree.ElementTree as ET
import codecs
import json
import re

In [2]:
def get_root(fname):
    tree = ET.parse(fname)
    return tree.getroot()
input_ankara = "ankara_turkey.osm"

In [3]:
def count_tags(root,tag):
    tag_count = 0
    for i in root.findall(tag):
        tag_count += 1
    return tag_count
 

In [32]:
root = get_root(input_ankara)

In [33]:
for i in ['node', 'way', 'relation' ]:
    print i, 'count: ', count_tags(root,i)

node count:  638467
way count:  92056
relation count:  220


In [29]:
ct= 0 
user = {}
for i in root.findall('node'):
    print i.attrib 
    try:
       user[i.attrib['user']] += 1
    except:
        user[i.attrib['user']] = 1
        
    if ct >5:
        break
    ct +=1

NameError: name 'root' is not defined

In [98]:
abbrevations = ['Cad.', '[cC]d']
address_tags = set()

In [402]:
ct = 0
rel = {}
for _,j in ET.iterparse("ankara_turkey.osm"):
    if j.tag == 'node':
        ct += 1
                    

In [403]:
ct

638467

In [139]:
for i,v in rel.iteritems():
    #print i,v
    print i.encode('utf-8')

## Problems ##
1) Abbreviated street, avenue names (e.g. 'Cad.', 'Cd', 'Sk.')

2) Use of multiple road/street names in the street name colum, such as  ('addr:street':'Barbaros Mah. Dereboyu Cad. Fulye Sok.') (which is Barbaros District Dereboyu Avenue Fulye Street)

3) Capitalization problems (e.g. 'cad.' , 'muvakkithane' , 'ULUBATLI HASAN SOKAK') 

4) Postcodes having more than 5 digits (i.e. 364303), or not starting with 34 (i.e. 41400)

In [392]:
global k 
k=0

In [4]:
def shape_element(element):
    node = {}
    lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
    problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

    if element.tag == "node" or element.tag == "way" :
        node['type'] = element.tag
        node['id'] =  element.attrib['id']
        
 
        try:          
            node['created'] = { 'timestamp' : element.attrib['timestamp'], 'version': element.attrib['version'],
                               'changeset': element.attrib['changeset'], 'user':element.attrib['user'], 
                               'uid': element.attrib['uid']}
            node['pos'] = [float(element.attrib['lat']), float(element.attrib['lon'])]
            node['visible'] =  element.attrib['visible']
            
            
  
            
        except:
            pass
        
        for i in element:
        
            if element.tag=="way":
                try:
                    node.setdefault('node_refs',[]).append(i.attrib['ref'])        #http://stackoverflow.com/questions/12905999/python-dict-how-to-create-key-or-append-an-element-to-key
                except:
                    pass
            
            try:
                if "addr" in i.attrib['k']:
                    addr, field = lower_colon.match(i.attrib['k']).group().split(':') 
                    length = len(lower_colon.match(i.attrib['k']).group().split(':'))
                    v = i.attrib['v']

                    if  length <= 2 and addr=='addr' and problemchars.match(v) == None:   
                        if field == 'postcode':
                            v = correct_postcode(v)

                        node.setdefault('address',{}).update({ field: cleanup(v).title()})
                else:
                     
                    if i.attrib['k'] != 'type': #some values have <tag k = "type" v=...>
                        node[i.attrib['k']] = i.attrib['v']
            except:
                pass
                    
            
                
        return node
    else:
  
        return None


In [5]:

def process_map(file_in, pretty = False):
    # You do not need to change this file
     
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            
            el  = shape_element(element)
            if el:
                 
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    print 'finish' 
    return data

In [ ]:
clean_ankara = process_map("ankara_turkey.osm")

In [54]:
print len(clean_ankara)

730523


In [275]:
def extend_abbrevations(word):
    
    abbrevations = { 'Cd.': 'Caddesi', 'Cad.':'Caddesi', 'Cd':'Caddesi', 'Sk.':'Sokak', 
                    'Mah.':'Mahallesi', 'Blv.':u'Bulvar\u0131', 'Bul.':u'Bulvar\u0131'}
    for i in word.split():
        try:
            word = word.replace(i,abbrevations[i])
        except:
            pass
    return word

In [276]:
cleanup('Atatürk blv.')

'Atat\xc3\xbcrk blv.'

In [448]:
def discard_multiples(way):
 
    if 'Sokak' in way and 'Mahallesi' in way:
        way= re.findall('Mahallesi (.*?)Sokak',way)[0]+ 'Sokak'
    if 'Caddesi'in way and 'Sokak' in way:
        way= re.findall('Caddesi (.*?)Sokak',way)[0]+ 'Sokak'
    return way
        

In [270]:
discard_multiples('ahmet yesevı Mahallesi guzel Sokak')

'guzel Sokak'

In [357]:
def correct_postcode(postcode):  #postcodes must start with 06, and must be 5 digits long
    if len(postcode) != 5:
        return None
    elif re.match(r'(06\d\d\d)',postcode)  == None:
        return None
    #return True
    return postcode
    

In [358]:
print correct_postcode(u'06121')

06121


In [339]:
re.match(r'(06...)','206131') == None

True

In [368]:
a=

In [371]:
'fafDa'.islower()

False